This notebook is written based on [this reference implementation](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb).

Other refs for model:
* https://stackoverflow.com/questions/65205582/how-can-i-add-a-bi-lstm-layer-on-top-of-bert-model
* https://discuss.pytorch.org/t/how-to-connect-hook-two-or-even-more-models-together/21033
* https://pytorch.org/tutorials/beginner/transformer_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Other refs for torchtext:
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-ii-f146c8b9a496
* http://anie.me/On-Torchtext/

# Imports and setup

In [2]:
import pandas as pd
import numpy as np
import os
import random
random.seed(1)
import re
import time

# Data processing.
import dataset # dataset.py
import torch
from torchtext.legacy import data 

# Model.
import models # models.py
import torch.nn as nn
import torch.optim as optim
from transformers import DistilBertModel, DistilBertTokenizer

# Training.
import training # training.py
from sklearn.model_selection import KFold
import utils # utils.py

# Visualization.
import matplotlib.pyplot as plt

# If you make a code change that doesn't get picked up by
# Jupyter notebook, try reloading like below:
# import imp
# imp.reload(training)

# Load a pre-trained BERT model

In [3]:
WEIGHTS_NAME = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(WEIGHTS_NAME)
bert = DistilBertModel.from_pretrained(WEIGHTS_NAME)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Read the data

In [6]:
data_df = dataset.get_multiple_datasets([0,1,2], 'Creativity_Combined', shuffle=True)

In [7]:
# For prototype purposes:
# split into train, test sets. (Train set will be further split into 
# train+validation sets, via k-fold CV.)
train_df = data_df[:1000]
test_df = data_df[1000:] # roughly 190 test examples set aside

# write them to CSV files
train_df.to_csv('ktrain.csv', index=False, header=False)
test_df.to_csv('ktest.csv', index=False, header=False)

## Preprocessing and transform into torchtext Dataset format.

From what I understand, some preprocessing is done when data.Field() is applied.

In [18]:
INIT_TOKEN_IDX = tokenizer.cls_token_id
EOS_TOKEN_IDX = tokenizer.sep_token_id
PAD_TOKEN_IDX = tokenizer.pad_token_id
UNK_TOKEN_IDX = tokenizer.unk_token_id

# BERT input can be at most 512 words
MAX_INPUT_LENGTH = tokenizer.max_model_input_sizes[WEIGHTS_NAME]

# Apply tokenization and some preprocessing steps to the input sentence.
# Namely, this trims examples down to MAX_INPUT_LENGTH. (There is a -2 
# since the [CLS] and [SEP] tokens will be added)
def tokenize_and_cut(sentence):
  sentence = sentence.replace('/', '') # remove slashes
  tokens = tokenizer.tokenize(sentence) 
  tokens = tokens[:MAX_INPUT_LENGTH-2]
  return tokens

# text_fields defines preprocessing and handling of the text of an example.
text_fields = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = INIT_TOKEN_IDX, # add [CLS] token
                  eos_token = EOS_TOKEN_IDX, # add [SEP] token
                  pad_token = PAD_TOKEN_IDX,
                  unk_token = UNK_TOKEN_IDX)

# label_fields defines how to handle the label of an example.
# for regression, we do not need to build a vocabulary.
label_fields = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)
all_fields = [('text', text_fields), ('label', label_fields)] # must match order of cols in csv

train_dataset, test_dataset = data.TabularDataset.splits(
  path='', # path='' because the csvs are in the same directory
  train='ktrain.csv', test='ktest.csv', format='csv',
  fields=all_fields  
)

In [15]:
print(data_df.head(1))

                                                 text  label
80  Performing a lottery among people who attend t...    3.6


In [20]:
# Transform train_dataset into an np array representation.
# This will be used for generating the K folds.
train_exs_arr = np.array(train_dataset.examples)

# Training pipeline begins here


## Define parameters

In [21]:
# Model parameters
OUTPUT_DIM = 1
DROPOUT = 0.2 # This is the distilbert default

# Training parameters
N_SPLITS = 5 # For cross validation
BATCH_SIZE = 8 # This is the distilbert default
N_EPOCHS = 3 # This is the distilbert default

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.MSELoss(size_average=False)
criterion = criterion.to(device)

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


## The cell where it actually trains!

In [26]:
# The main training loop
def launch_experiment(train_data_df, n_splits=5):
  # valid_corrs = np.empty(n_splits)
  best_valid_loss = float('inf') 
  
  kf = KFold(n_splits=n_splits)
  fold = 0
  for train_index, valid_index in kf.split(train_data_df):
    print('training on fold {}'.format(fold))
    train_data = data.Dataset(train_exs_arr[train_index], all_fields)
    valid_data = data.Dataset(train_exs_arr[valid_index], all_fields)
    
    # Initialize a new model each fold.
    # https://ai.stackexchange.com/questions/18221/deep-learning-with-kfold-cross-validation-with-epochs
    # https://stats.stackexchange.com/questions/358380/neural-networks-epochs-with-10-fold-cross-validation-doing-something-wrong
    model = models.BERTLinear(bert,
                              OUTPUT_DIM,
                              DROPOUT)
    optimizer = optim.Adam(model.parameters())
    model = model.to(device)

    train_iterator, valid_iterator = training.get_iterators(train_data, valid_data, BATCH_SIZE, device)

    for epoch in range(N_EPOCHS):
      start_time = time.time()
      train_loss, train_corr = training.train(model, train_iterator, optimizer, criterion)
      valid_loss, valid_corr = training.evaluate(model, valid_iterator, criterion)
      # valid_corrs[fold] will end up holding the valid_corr from the last epoch for this fold
      # is this what we want?
      # valid_corrs[fold] = valid_corr
      end_time = time.time()
      epoch_mins, epoch_secs = utils.epoch_time(start_time, end_time)

      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        # Save the weights of the model with the best valid loss
        print('updating saved weights of best model')
        torch.save(model.state_dict(), "best_valid_loss.pt")

      print(f'Epoch: {epoch:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\t Train Loss: {train_loss:.3f} | Train Corr: {train_corr:.2f}')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Corr: {valid_corr:.2f}')
    
  fold += 1
  
  return best_valid_loss # , valid_corrs

In [27]:
best_valid_loss = launch_experiment(train_exs_arr)
print('best validation loss: {}'.format(best_valid_loss))
# print('validation correlations for each fold: {}'.format(valid_corrs))
# print('average of validation correlations: {}'.format(np.mean(valid_corrs)))

training on fold 0
updating saved weights of best model
Epoch: 00 | Epoch Time: 0m 46s
	 Train Loss: 7.945 | Train Corr: -0.04
	 Val. Loss: 18.523 |  Val. Corr: 0.01
updating saved weights of best model
Epoch: 01 | Epoch Time: 0m 45s
	 Train Loss: 5.523 | Train Corr: -0.01
	 Val. Loss: 10.152 |  Val. Corr: -0.03
Epoch: 02 | Epoch Time: 0m 46s
	 Train Loss: 5.939 | Train Corr: 0.02
	 Val. Loss: 13.532 |  Val. Corr: -0.02
training on fold 3
updating saved weights of best model
Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 8.778 | Train Corr: -0.02
	 Val. Loss: 8.787 |  Val. Corr: 0.01
updating saved weights of best model
Epoch: 01 | Epoch Time: 0m 48s
	 Train Loss: 6.524 | Train Corr: 0.02
	 Val. Loss: 6.050 |  Val. Corr: -0.02
Epoch: 02 | Epoch Time: 0m 48s
	 Train Loss: 6.565 | Train Corr: 0.01
	 Val. Loss: 7.477 |  Val. Corr: -0.01
training on fold 6
updating saved weights of best model
Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 8.255 | Train Corr: 0.05
	 Val. Loss: 5.256 |  Val. Cor

# Test the trained model on held-out dataset.

In [28]:
# Get a test iterator
test_iterator = training.get_iterator(test_dataset, BATCH_SIZE, device)

In [32]:
import imp
imp.reload(training)

# load the best model saved
model = models.BERTLinear(bert, OUTPUT_DIM, DROPOUT)
model.load_state_dict(torch.load("best_valid_loss.pt"))
model.to(device)
model.eval()
test_loss, test_corr = training.evaluate(model, test_iterator, criterion, debug=True)
print(test_loss)
print(test_corr)

predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([3.7778, 4.1111, 4.5000, 2.4722, 4.2778, 5.0556, 4.5278, 3.0278],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([3.6667, 4.0278, 2.7500, 3.8056, 3.0556, 3.0278, 3.8333, 2.5000],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([1.6944, 3.6667, 3.3611, 3.6111, 4.6389, 2.6111, 3.1944, 4.5556],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([4.2500, 5.1111, 3.6944, 4.2500, 3.8889, 3.1111, 3.1389, 4.0278],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true label

/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([2.6111, 5.1389, 4.1667, 3.2500, 3.3611, 3.6667, 3.8611, 2.7222],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([2.0278, 3.9167, 2.5833, 4.1111, 4.2222, 2.9167, 4.4722, 3.8611],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([3.0278, 3.9444, 3.9167, 3.6389, 4.1111, 4.7778, 2.9167, 3.0278],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true labels: tensor([3.8056, 4.1111, 3.8889, 4.1944, 2.3889, 4.0278, 5.2500, 3.8611],
       device='cuda:0')
predictions: tensor([3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724, 3.8724],
       device='cuda:0')
true label

# Misc other stuff

Link to the trainer class: https://huggingface.co/transformers/main_classes/trainer.html



Default training arguments: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments

Batch size per device: 8

Epoch: 3



This should be the model I used to generate my initial results: https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
"DistilBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks."